<a href="https://colab.research.google.com/github/zeynep68/CIFAR100/blob/main/ResNet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch.nn import Sequential, Conv2d, BatchNorm2d, ReLU, MaxPool2d, AvgPool2d, Softmax, Flatten
from torchvision import transforms
from torch.utils.data import DataLoader, sampler
from torchvision.datasets import CIFAR100 
from torch.optim.lr_scheduler import MultiStepLR

# Load and prepare data



In [124]:
train_mean = torch.Tensor([0.5072, 0.4867, 0.4411])
train_std = torch.Tensor([0.2668, 0.2559, 0.2756])
test_mean = torch.Tensor([0.5142, 0.4928, 0.4467])
test_std = torch.Tensor([0.2707, 0.2597, 0.2794])

In [125]:
BATCH_SIZE = 256
transform_train = transforms.Compose([transforms.ToTensor(), transforms.Normalize(train_mean, train_std)])
transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize(test_mean, test_std)])

In [126]:
train = CIFAR100(root='./trainset', train=True, download=True, transform=transform_train)
test = CIFAR100(root='./testset', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [127]:
len(train), len(test)

(50000, 10000)

In [128]:
NUM_TRAIN = 49000
train_loader = DataLoader(train, batch_size=BATCH_SIZE, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN))) # 391
val_loader = DataLoader(train, batch_size=BATCH_SIZE, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))
test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

# ResNet


In [129]:
class ConvolutionalModule(nn.Module):
  def __init__(self, input_channel=64, output_channel=128):
    super(ConvolutionalModule, self).__init__()
    self.layer1 = Sequential(
                    Conv2d(in_channels=input_channel, out_channels=output_channel, kernel_size=3, stride=1, padding=1),
                    BatchNorm2d(output_channel), # über channels des outputs
                    ReLU()
                  )
    self.layer2 = Sequential(
                    Conv2d(in_channels=output_channel, out_channels=output_channel, kernel_size=3, stride=2, padding=1),
                    BatchNorm2d(output_channel), # über channels des outputs
                    ReLU()
                )    

    self.x_shortcut = Sequential(
                        Conv2d(in_channels=input_channel, out_channels=output_channel, kernel_size=1, stride=2, padding=0),
                        BatchNorm2d(output_channel),
                        ReLU()
                      )        

  def forward(self, x):
    return self.x_shortcut(x) + self.layer2(self.layer1(x))

In [130]:
class IdentityModule(nn.Module):
  def __init__(self, num_channel):
    super(IdentityModule, self).__init__()
    self.layer1 = Sequential(
                    Conv2d(in_channels=num_channel, out_channels=num_channel, kernel_size=3, stride=1, padding=1),
                    BatchNorm2d(num_channel), # über channels des outputs
                    ReLU()
                  )
    self.layer2 = Sequential(
                    Conv2d(in_channels=num_channel, out_channels=num_channel, kernel_size=3, stride=1, padding=1),
                    BatchNorm2d(num_channel),
                    ReLU()
                )          

  def forward(self, x):
    return x + self.layer2(self.layer1(x))

In [131]:
class ResNet34(nn.Module):
  def __init__(self):
    super(ResNet34, self).__init__()
    self.conv1 = Sequential(
                         Conv2d(in_channels=3, out_channels=64, kernel_size=7 ,stride=2, padding=0),
                         BatchNorm2d(64),
                         ReLU(),
                         MaxPool2d(kernel_size=3, stride=2)
                       )
    
    self.conv2 = Sequential(
                   IdentityModule(64),
                   IdentityModule(64),
                   IdentityModule(64)
                 )

    self.conv3 = Sequential(
                   ConvolutionalModule(),
                   IdentityModule(128),
                   IdentityModule(128),
                   IdentityModule(128)
                 )
    
    self.conv4 = Sequential(
                  ConvolutionalModule(128, 256),
                  IdentityModule(256),
                  IdentityModule(256),
                  IdentityModule(256),
                  IdentityModule(256),
                  IdentityModule(256)
                )
    
    self.conv5 = Sequential(
                   ConvolutionalModule(256, 512),
                   IdentityModule(512),
                   IdentityModule(512)
                 )

    self.output_layer = Sequential(
                          AvgPool2d(kernel_size=1),
                          Flatten(),
                          Softmax(dim=1)
                        )
    

  def forward(self, x):
    y = self.conv1(x)
    y = self.conv2(y)
    y = self.conv3(y)
    y = self.conv4(y)
    y = self.conv5(y)
    y = self.output_layer(y)
    return y

# Function to check accuracy

In [132]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('using device:', DEVICE)

using device: cuda


In [133]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print('Checking accuracy on validation set')
  else:
    print('Checking accuracy on test set') 
  model.eval() 

  num_correct = 0
  num_samples = 0

  with torch.no_grad():
    for images, labels in loader: # durchläuft 391 mal (mit jeweils 128 Bildern)
      images = images.to(device=DEVICE)
      labels = labels.to(device=DEVICE)
      scores = model(images)
      _, predictions = scores.max(1)
      num_correct += (predictions == labels).sum()
      num_samples += predictions.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

# Training

In [134]:
print_every = 100

In [135]:
def train(model, optimizer, epochs=120):

  model = model.to(device=DEVICE)
  for epoch in range(epochs):
    scheduler.step()
    print('Epoch:', epoch,'LR:', scheduler.get_lr())
    for t, (images, labels) in enumerate(train_loader): # durchläuft 391 mal (mit jeweils 128 Bildern)
      model.train() 
      images = images.to(device=DEVICE)
      labels = labels.to(device=DEVICE)

      scores = model(images)
      loss = F.cross_entropy(scores, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if t % print_every == 0:
        print('Iteration %d, loss = %.4f' % (t, loss.item()))
        check_accuracy(val_loader, model)
        print()

In [145]:
learning_rate = 1e-3
model = ResNet34()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = MultiStepLR(optimizer, milestones=[40,80, 100], gamma=0.1)

In [146]:
train(model, optimizer)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:396: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Epoch: 0 LR: [0.001]
Iteration 0, loss = 6.2385
Checking accuracy on validation set
Got 0 / 1000 correct (0.00)

Iteration 100, loss = 6.1894
Checking accuracy on validation set
Got 40 / 1000 correct (4.00)

Epoch: 1 LR: [0.001]
Iteration 0, loss = 6.1415
Checking accuracy on validation set
Got 77 / 1000 correct (7.70)

Iteration 100, loss = 6.1120
Checking accuracy on validation set
Got 102 / 1000 correct (10.20)

Epoch: 2 LR: [0.001]
Iteration 0, loss = 6.0958
Checking accuracy on validation set
Got 114 / 1000 correct (11.40)

Iteration 100, loss = 6.0490
Checking accuracy on validation set
Got 134 / 1000 correct (13.40)

Epoch: 3 LR: [0.001]
Iteration 0, loss = 6.0833
Checking accuracy on validation set
Got 150 / 1000 correct (15.00)

Iteration 100, loss = 6.0634
Checking accuracy on validation set
Got 132 / 1000 correct (13.20)

Epoch: 4 LR: [0.001]
Iteration 0, loss = 6.0421
Checking accuracy on validation set
Got 157 / 1000 correct (15.70)

Iteration 100, loss = 6.0305
Checking a

KeyboardInterrupt: ignored

# Evaluating